In [47]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
import os
import numpy as np
import pandas as pd
from collections import Counter
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
stop = stopwords.words('english')
import string
punctuations = list(string.punctuation)
stop = stop + punctuations

In [48]:

def ModPosTag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# def simple_clean(words):
#     output_words = [w.lower() for w in words if not w.lower() in stop]
#     return output_words

def simple_clean(words):
    output_words = [w for w in words if not w in stop]
    return output_words


# def complex_clean(words):
#     output_words = []
#     for w in words:
#         if w.lower() not in stop:
#             pos = pos_tag([w])
#             clean_w = lemmatizer.lemmatize(w, pos = ModPosTag(pos[0][1]))
#             output_words.append(clean_w.lower())
#     return output_words

# remove lemetize
def complex_clean(words):
    output_words = []
    for w in words:
        if w not in stop:
            pos = pos_tag([w])
            clean_w = lemmatizer.lemmatize(w, pos = ModPosTag(pos[0][1]))
            output_words.append(clean_w)
    return output_words


def read_words(words_dir):

    files = [os.path.join(words_dir, fi) for fi in os.listdir(words_dir)]

    #features_matrix = np.zeros((len(files), 3000))
    docID = 0;
    all_words = []
    all_words_heading = []
    major_words_abstract = []
    for fil in files:
        with open(fil) as fi:
            words_abstract = []
            for i, line in enumerate(fi):
                if (i == 0):
                    temp = line.split()
                    temp1 = simple_clean(temp)
                    
                    words_abstract += temp1
                    all_words_heading += temp1
                else:
                    temp = line.split()
                    temp1 = complex_clean(temp)
                    print(fil)
                    print()
                    print(temp1)
                    words_abstract += temp1
                    all_words += temp1

            major_words_abstract.append(words_abstract)
            len(all_words)
    return major_words_abstract, all_words, all_words_heading

def tag_updater(df, variable_code):
    for key, values in variable_code.items():
        temp = str(key)
        area = variable_code[key]
        for val in area:
            df.at[val-1, temp] = 1

    return df


def read_tags(words_dir):
    files = [os.path.join(words_dir, fi) for fi in os.listdir(words_dir)]

    # features_matrix = np.zeros((len(files), 3000))
    docID = 0;
    tagss = set()
    for fil in files:
        docID+=1
        with open(fil) as fi:
            for i, line in enumerate(fi):
                temp = line.split(',')
                tagss.update(temp)
    return docID, tagss




In [49]:
mydict = {}

def read_files(words_directory, tags_directory):

    wo = [os.path.join(words_directory, wi) for wi in os.listdir(words_directory)]
    ta = [os.path.join(tags_directory, ti) for ti in os.listdir(tags_directory)]
    ctr=0
    for (ab, t) in zip(wo, ta):
        ctr +=1
        with open(t) as su:
            for i, line in enumerate(su):
                tag = line.split(',')
                for q in tag:
                    if q not in mydict:
                        mydict[q] = []
                    mydict[q].append(ctr)

    return mydict,ctr

def extract_features(Words, features):
    feature_matrix = np.zeros((len(Words), len(features)))
    docID = 0
    for doc in Words:
        for word in doc:
            for i in range(len(features)):
                if features[i] == word:
                    wordID = i
                    feature_matrix[docID, wordID] += 1
        docID = docID + 1
    np.shape(feature_matrix)
    print(feature_matrix)
    return feature_matrix

In [50]:
words_dir = 'abstract'
tags_directory = 'tags'
Words, all_words, all_words_heading = read_words(words_dir)


In [51]:
cou, Tags = read_tags(tags_directory)
#df = pd.DataFrame(Tags)
Words = np.array(Words)
# exploring frequency of all words not in heading
import nltk
freq = nltk.FreqDist(all_words + all_words_heading)
common = freq.most_common(3000)
common = list(common)

features = []
features += [w[0] for w in common]
features += [w for w in all_words_heading if w not in common]
print(len(common))
print(len(features))


6
132


In [52]:


variable_code, ctr = read_files(words_dir, tags_directory)
columns = list(Tags)
rows = list(range(ctr))
dataFrame = np.zeros((len(rows), len(columns)))
#df = tag_updater(dataFrame, variable_code, columns)

df = pd.DataFrame(data = dataFrame, index = rows, columns = columns, dtype='int64')
# print(variable_code)
Y = tag_updater(df, variable_code)
#print(df.at[1, ' Machine learning '])
# for x in df['Computers']:
#     print(x)

display(Y)
Y.fillna(0, inplace=True)

X = extract_features(Words, features)
X_train = pd.DataFrame(data = X, index = rows, columns = features, dtype='int64')


# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(X_train, Y)
# predict
#predictions = classifier.predict(X_train)
#print(predictions.toarray())
#print(accuracy_score(Y, predictions))

,DS,Algorithm,AI,Machine Learning
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0
5,1,1,0,0
6,1,1,0,0
7,1,1,0,0
8,1,1,0,0
9,1,1,0,0


[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


BinaryRelevance(classifier=GaussianNB(priors=None, var_smoothing=1e-09),
        require_dense=[True, True])

In [55]:
words_dir = 'testing'
Words, waste1, waste2 = read_words(words_dir)
X = extract_features(Words, features)
rows = list(range(1))
X_test = pd.DataFrame(data = X, columns = features, dtype='int64')


[[1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.
  1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.
  1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.
  1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.
  1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.
  1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.]]


In [56]:
display(X_test)
predictions = classifier.predict(X_test)

,Support,Vector,Machines,Classify,Learning,Systems,Support,Vector,Machines,Classify,...,Machines,Classify,Learning,Systems,Support,Vector,Machines,Classify,Learning,Systems
0,1,1,1,0,0,0,1,1,1,0,...,1,0,0,0,1,1,1,0,0,0


C:\Users\Mohit\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\Mohit\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\Mohit\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\Mohit\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\Mohit\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\Mohit\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\Moh

In [57]:
display(predictions)
predictions.shape[0]

<1x4 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Column format>

1

In [58]:
from scipy.sparse import csc_matrix
print(type(predictions))
a = predictions.nonzero()
#a.row[a.data]

<class 'scipy.sparse.csc.csc_matrix'>


In [59]:
def find_index(self):
    A = self.tocoo()
    nz_mask = A.data != 0
    return (list(A.col[nz_mask]))

In [60]:
print((predictions.shape[0], len(columns)))
dataFrame5 = np.zeros((predictions.shape[0], len(columns)))
ct = 0
for i in predictions:
    b = find_index(i)
    for j in b:
        dataFrame5[ct,j] +=1
    ct+=1
print(dataFrame5)
for i in dataFrame5:
    print(len(i))

(1, 4)
[[0. 0. 0. 0.]]
4


In [61]:
print(type(dataFrame))

converting = pd.DataFrame(data = dataFrame5, columns = columns, dtype='int64')
display(converting)

<class 'numpy.ndarray'>


,DS,Algorithm,AI,Machine Learning
0,0,0,0,0


In [40]:
df.to_csv('file_predicted_skills.csv', sep='\t', encoding='utf-8')

In [41]:
we_have = []
print('------------------------TECHNICAL SKILLS REQUIRED-----------------------------')
for i in b:
    print(columns[i])
    we_have.append(columns[i])
    
#we_have.append('artificial intelligence')
#we_have.append('Machine learning algorithms')
#we_have.append('Support vector machines')
#we_have.append('Kernel')

print(we_have)

------------------------TECHNICAL SKILLS REQUIRED-----------------------------
[]


In [51]:
#this is keyword extractor

In [52]:
#next comes the wiki reader

In [20]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import nltk
import pickle


lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''


def load_skill_set():

    skill_dict = dict()
    pick_file = open("final_skill_dict.pickle", "rb")
    skill_dict = pickle.load(pick_file)
    pick_file.close()
    return skill_dict


skill_dict = load_skill_set()


def get_skills(skill_dic,topic):

    # print("getting skill set for ",topic)
    skill_set = set()
    for word in topic.split(' '):
        # print("for word=",word)
        if word.strip().isalpha():
            word = word.strip().lower()
            wordnet_pos = get_wordnet_pos(nltk.pos_tag([word])[0][1])
            if wordnet_pos == '':
                word = lemmatizer.lemmatize(word)
            else:
                word = lemmatizer.lemmatize(word, wordnet_pos)

            try:
                curr_skill_set = skill_dic[word]
                # print("skill set=",curr_skill_set)
                if len(skill_set) == 0:
                    skill_set = curr_skill_set
                else:
                    skill_set = skill_set.intersection(curr_skill_set)

                #print("intersection=",skill_set)
            except KeyError as e:
                print("no skill set found",e)

    return skill_set


print(get_skills(skill_dict,'opportunity'))


{'identifying and seizing fast breaking opportunities', 'evaluation of commercial opportunities', 'identification and evaluation of market opportunities', 'opportunity management'}


In [21]:
import urllib.request
from bs4 import BeautifulSoup


def get_page(topic):
    domain = "https://en.wikipedia.org"
    html = urllib.request.urlopen("https://en.wikipedia.org/w/index.php?search="+topic.replace(' ','+')+"&title=Special%3ASearch&go=Go")
    soup = BeautifulSoup(html, features="lxml")
    first_result = soup.find(attrs={"data-serp-pos": "0"})
    if first_result is None:
        print('page-found')
        return soup
    href = first_result.get('href')
    print('opening first-result')
    html = urllib.request.urlopen(domain+href)
    soup = BeautifulSoup(html, features="lxml")
    return soup


def get_first_para(topic):
    soup = get_page(topic)
    text_section = soup.find(attrs={'class': 'mw-parser-output'})
    text = ''
    for child in text_section.children:
        # print('for tag', child.name, child)
        try:
            if child is not None:
                if child.name == 'p':
                    text += child.text.lower()
                elif child.name == 'div' and 'toc' in child['class']:
                    break
        except Exception as e:
            print("exception", e)
    return text

vo_set = set()

In [22]:
import re
# cleaning we _have 
dore = []
for x in we_have:
    regex = re.compile('[^a-zA-Z]')
    #First parameter is the replacement, second parameter is your input string
    #regex.sub('', x)
    dore.append(re.sub("[^a-zA-Z_ ]*", "", x))

for y in dore:
    print(y)

In [23]:
import csv
with open('tech2.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(dore)

In [24]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize


if __name__ == '__main__':

    #print(get_first_para('computer vision'))
    for original in dore: 
        print(original)
        w1=get_first_para(original)
        w2=word_tokenize(w1)
        
        print(w1)

        for word in w2:                                                                 
            if word not in stop and word.__len__()>3 and word.isalpha():
                w3 = get_skills(skill_dict, word)
                for x in w3:
                    vo_set.add(x)
                
                


In [25]:
vo_set

set()

In [26]:
from nltk.stem import WordNetLemmatizer
import pickle


lemmatizer = WordNetLemmatizer()


def load_skill_set2():

    skill_dict = dict()
    pick_file = open("final_skill_dict_2.pickle", "rb")
    skill_dict = pickle.load(pick_file)
    pick_file.close()
    return skill_dict


def get_skill2(skill_dic,topic):
    return skill_dic[ lemmatizer.lemmatize(topic.strip()) ]


# skill_dict = make_skill_dict()
skill_dict2 = load_skill_set2()

print(get_skill2(skill_dict2,'procurement management'))



facilitation skills


In [60]:

print(len(vo_set))

544


In [61]:
new_set = set()
for i in vo_set:
    if i in skill_dict2:
        new_set.add(get_skill2(skill_dict2,i))

len(new_set)

97

In [62]:
print('--------------NON- TECHNICAL SKILLS---------------')
for word in new_set:
    print(word)
my_list2 = list(new_set)

--------------NON- TECHNICAL SKILLS---------------
communication skills
scope management
emotional intelligence
marketing communications
sales pipeline management
negotiation
technical skills
cultural competence
sales operations
visual abilities
core project management skills
benefits management
quality management
procurement management
speaking skills
sales management
information modeling & design
selling skills
networking
program management skills
core executive leadership skills
basic leadership skills
risk management
executive management
core management skills
creative skills
problem solving
time management
entrepreneurial skills
technology (for it project management)
idea formation
program lifecycle
management
market research
targeted communication
human resources management
sales strategy
brand management
writing
persuasion techniques
influencing to negotiate
using emotions
decision making
hard bargaining
sale
influencing
human resources
managerial finance
finance
information vis

In [63]:
import csv
with open('nontech2.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(my_list2)

In [64]:
# trying out reading just the extract for the technical skills
import os
files = [os.path.join('testing', fi) for fi in os.listdir('testing')]
my_text = []
from nltk.tokenize import word_tokenize
for fil in files:
    with open(fil) as fi:
            line = fi.read()
            temp= word_tokenize(line)

In [65]:
temp2 = list(temp)
my_text =  [w.lower() for w in temp2 if not w.lower() in stop]

In [66]:
my_text

['determination',
 'vocational',
 'fields',
 'machine',
 'learning',
 'algorithm',
 'importance',
 'vocational',
 'technical',
 'training',
 'growing',
 'day',
 'day',
 'parallel',
 'developing',
 'technology',
 'inevitable',
 'utilise',
 'opportunities',
 'presented',
 'information',
 'communication',
 'technologies',
 'order',
 'determine',
 'vocational',
 'fields',
 'vocational',
 'technical',
 'training',
 'efficient',
 'manner',
 'respect',
 'possible',
 'create',
 'efficient',
 'tool',
 'compared',
 'current',
 'methods',
 'utilising',
 'machine',
 'learning',
 'artificial',
 'intelligence',
 'model',
 'energy',
 'applications',
 'predicts',
 'events',
 'future',
 'depending',
 'past',
 'experiences',
 'current',
 'study',
 'software',
 'developed',
 'ensures',
 'system',
 'learns',
 'successful',
 'unsuccessful',
 'choices',
 'made',
 'past',
 'applying',
 '“',
 'naive',
 'bayes',
 '”',
 'algorithm',
 'machine',
 'learning',
 'algorithm',
 'data',
 'collected',
 'concerning',
 '

In [67]:
q_set = set()
for q in my_text:
    if q in skill_dict:
        veu = get_skills(skill_dict,q)
        for qw in veu:
            q_set.add(qw)

In [68]:
for qw in q_set:
    print(qw)

communication skills
emotional intelligence
creative direction
digital business strategy
communications to stakeholders
managing emotions
applying economic models
listening.
visual expression of concepts
publicity
marketing communications
intuition
creative questioning
flow charts
graphs
creating and maintaining a high performance culture
project metric development and communications
effective greetings
friendliness
technology savvy
interviewing
whiteboarding
communications
targeting information to an audience
statistical analysis
negotiation
empathy
developing presentations
timelines
using project management tools
designing graphics
community engagement
quick-wittedness
reporting & communication
communications to executive leadership
communication plans
cultural competence
managing a portfolio of it investments
research
applying data to decisions
managing training
visualization standards
fluid intelligence
marketing intelligence management
selective listening
technology trend awarenes

In [69]:
len(q_set)

256

In [70]:
qq_set = set()
for i in q_set:
    qq_set.add(get_skill2(skill_dict2,i))

In [71]:
temp = []
for ele in qq_set:
    temp = temp + (word_tokenize(ele))

In [95]:
dict123 = {}
my_final = {}
my_final['others'] = set()
print(temp)
for i in temp:
    if i in dict123:
        dict123[i] = dict123[i]+1
    else:
        dict123[i] = 1
for ele in qq_set:
    w = word_tokenize(ele)
    for i in w:
        if dict123[i]>5:
            if i in my_final:
                my_final[i].add(ele)
            else:
                my_final[i] = set()
                my_final[i].add(ele)
        if dict123[i]<2:
            my_final['others'].add(ele)
len(qq_set)
len(my_final)

['communication', 'skills', 'presenting', 'visual', 'information', 'cost', 'management', 'scope', 'management', 'emotional', 'intelligence', 'finance', 'information', 'visualization', 'marketing', 'communications', 'business', '&', 'product', 'development', 'strategic', 'program', 'management', 'core', 'executive', 'leadership', 'skills', 'managing', 'visual', 'information', 'sales', 'pipeline', 'management', 'risk', 'management', 'executive', 'management', 'core', 'management', 'skills', 'presentation', 'skills', 'soft', 'skills', 'applying', 'innovation', 'techniques', 'technical', 'skills', 'stakeholder', 'management', 'marketing', 'strategy', '&', 'planning', 'time', 'management', 'entrepreneurial', 'skills', 'project', 'reporting', '&', 'communication', 'technology', '(', 'for', 'it', 'project', 'management', ')', 'change', 'management', 'professional', 'communication', 'business', 'execution', 'cultural', 'competence', 'sales', 'operations', 'idea', 'formation', 'program', 'lifec

4

In [96]:
my_final

{'communication': {'communication',
  'communication skills',
  'leadership of visual communication',
  'professional communication',
  'reporting & communication',
  'targeted communication',
  'visual communication'},
 'management': {'brand management',
  'change management',
  'core management skills',
  'cost management',
  'executive management',
  'human resources management',
  'integration management',
  'management',
  'procurement management',
  'risk management',
  'sales management',
  'sales pipeline management',
  'scope management',
  'stakeholder management',
  'strategic program management',
  'technology (for it project management)',
  'time management'},
 'others': {'applying innovation techniques',
  'brand management',
  'business & product development',
  'business execution',
  'change management',
  'cost management',
  'cultural competence',
  'decision making',
  'emotional intelligence',
  'entrepreneurial skills',
  'finance',
  'governance',
  'human resour

In [97]:
### --------    Anupams dictionary  --------  ###


# from nltk.stem import WordNetLemmatizer
# import pickle
# import xlrd


# lemmatizer = WordNetLemmatizer()

# # print(sheet.cell_value(0,0))


# def make_skill_dict_final():

#     file = ("test_files/skills_anupam.xlsx")

#     wb = xlrd.open_workbook(file)
#     sheet = wb.sheet_by_index(0)

#     # print(len(lines))
#     skill_dict = {}

#     root_skill = ''
#     for i in range(sheet.nrows):
#         if sheet.cell_value(i,0) != '':
#             root_skill = lemmatizer.lemmatize(sheet.cell_value(i, 0).strip().lower())

#         for j in range(sheet.ncols):

#             if sheet.cell_value(i, j) != '':
#                 key = lemmatizer.lemmatize(sheet.cell_value(i, j).strip().lower())

#                 if key in skill_dict:
#                     skill_dict[key].add(root_skill)
#                 else:
#                     skill_dict[key] = {root_skill}

#     print("printing dict.......")
#     for key in skill_dict:
#         print(str(key).ljust(40) + str(skill_dict[key]).rjust(40))

#     pick_file = open("anupam_skill_dict.pickle","wb")
#     pickle.dump(skill_dict, pick_file)
#     pick_file.close()

#     return skill_dict


# def load_skill_set_final():

#     pick_file = open("anupam_skill_dict.pickle", "rb")
#     skill_dict = pickle.load(pick_file)
#     pick_file.close()
#     return skill_dict


# def get_skill_final(skill_dic,topic):
#     topic = lemmatizer.lemmatize(topic.strip().lower())
#     if topic in skill_dic:
#         return skill_dic[topic]
#     else:
#         return None


# # skill_dict = make_skill_dict()
# skill_dict_final = load_skill_set()

# print(get_skill_final(skill_dict_final,'motivate'))
# # while True:
# #     inp = input("topic:")
# #     print(get_skill(skill_dict,inp))

In [98]:
print('----------------Technical skills---------------')
for zzz in dore:
    print(zzz)
print()
print('----------------Technical skills---------------')
print()

for i in my_final:
    
    print('--------------------------------')
    print('             --', i , '--')
    print('---------------------------------')
    for j in my_final[i]:
        print(j, ',')
        

----------------Technical skills---------------
Machine learning 
 Machine learning algorithms 
 Support vector machines 

----------------Technical skills---------------

--------------------------------
             -- others --
---------------------------------
presenting visual information ,
cost management ,
scope management ,
emotional intelligence ,
finance ,
information visualization ,
marketing communications ,
business & product development ,
strategic program management ,
managing visual information ,
sales pipeline management ,
applying innovation techniques ,
risk management ,
presentation skills ,
soft skills ,
technical skills ,
stakeholder management ,
marketing strategy & planning ,
time management ,
entrepreneurial skills ,
reporting & communication ,
technology (for it project management) ,
business execution ,
cultural competence ,
sales operations ,
idea formation ,
program lifecycle ,
targeted communication ,
market research ,
interpersonal skills ,
human resource